# Step 1: Data cleaning and merge

In [2]:
import numpy as np
import pandas as pd
#import omdb

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

In [3]:
movies = pd.read_csv('data/movies.csv')
emmy = pd.read_csv('data/the_emmy_awards.csv')
oscar = pd.read_csv('data/the_oscar_award.csv')
movies_b = pd.read_csv('data/MoviesOnStreamingPlatforms.csv')
type_amazon= pd.read_csv('data/amazon_prime_titles.csv')
type_netflix = pd.read_csv('data/netflix_titles.csv')

In [4]:
#only load values that matches the title values from movies
all_titles_in_movies = movies['Title'].unique()

In [5]:
#np.where(all_titles_in_movies == 'The First King')
all_titles_in_movies.size

14858

In [6]:
#dowload this file from https://datasets.imdbws.com/
title_info = pd.DataFrame()
chunksize = 10 ** 5
for chunk in pd.read_csv("data/title.basics.tsv", delimiter='\t', chunksize=chunksize):
    #title_info=title_info.append (chunk)
    title_info=title_info.append (chunk[chunk.originalTitle.isin(all_titles_in_movies)])


In [7]:
#Checking for the number of unique values for each columns. 
#note that the title column has duplicate values. 
movies.nunique(axis=0)

Unnamed: 0            24664
ID                    24664
Title                 14858
Year                    109
Rating                    5
IMDb                     87
Rotten Tomatoes          85
Genre                    14
Netflix                   2
Amazon Prime Video        2
dtype: int64

In [8]:
#checking for non null values to identity columns that are can be potentially removed.
#the ratings column has a high number of null values, but this column is essential for our analysis.
#so will look to fill the null values with values from other sources

movies.isnull().sum()

Unnamed: 0                0
ID                        0
Title                     0
Year                      0
Rating                14635
IMDb                    420
Rotten Tomatoes           0
Genre                     0
Netflix                   0
Amazon Prime Video        0
dtype: int64

In [9]:
movies.head()

,Unnamed: 0,ID,Title,Year,Rating,IMDb,Rotten Tomatoes,Genre,Netflix,Amazon Prime Video
0,0,1,Terminator: Dark Fate,2019,18+,6.2,81,Action & Adventure,0,1
1,1,2,Gemini Man,2019,13+,5.7,74,Action & Adventure,0,1
2,2,3,Rambo: Last Blood,2019,18+,6.1,72,Action & Adventure,0,1
3,3,4,The Courier,2019,18+,4.9,50,Action & Adventure,0,1
4,4,5,Crawl,2019,18+,6.1,79,Action & Adventure,0,1


In [10]:
#identifying the duplicate title values. The number of duplicate rows match when added with the
# values of the number of titles availale. 9806+14858 = 24664
movies.loc[movies.Title.duplicated(),['ID','Title']]

,ID,Title
105,106,Love Live! The School Idol Movie
110,111,The Stolen Princess: Ruslan and Ludmila
132,133,Maya the Bee Movie
136,137,Ernest & Celestine
252,253,I'll See You in My Dreams
253,254,Diary of a Mad Black Woman
256,257,Last Flag Flying
257,258,Trees Lounge
260,261,Thank You for Smoking
265,266,Rudderless


In [11]:
#checking to see why the titles are duplicated.
#observed that the rows are duplicated to record the multiple genre the film belongs to or
#if the movie is offered in netflix and amazon. 
movies.loc[movies['Title'].str.contains('The Stolen Princess: Ruslan and Ludmila', na=False)]

,Unnamed: 0,ID,Title,Year,Rating,IMDb,Rotten Tomatoes,Genre,Netflix,Amazon Prime Video
38,38,39,The Stolen Princess: Ruslan and Ludmila,2018,NaN,6.1,56,Animation,0,1
110,110,111,The Stolen Princess: Ruslan and Ludmila,2018,NaN,6.1,56,Comedy,0,1
3949,3949,3950,The Stolen Princess: Ruslan and Ludmila,2018,NaN,6.1,56,Action & Adventure,0,1


In [12]:
#checking to see the the year range for which the titles are avaiable 
sorted(movies.Year.unique())

[1912,
 1913,
 1915,
 1916,
 1917,
 1918,
 1919,
 1920,
 1921,
 1922,
 1923,
 1924,
 1925,
 1926,
 1927,
 1928,
 1929,
 1930,
 1931,
 1932,
 1933,
 1934,
 1935,
 1936,
 1937,
 1938,
 1939,
 1940,
 1941,
 1942,
 1943,
 1944,
 1945,
 1946,
 1947,
 1948,
 1949,
 1950,
 1951,
 1952,
 1953,
 1954,
 1955,
 1956,
 1957,
 1958,
 1959,
 1960,
 1961,
 1962,
 1963,
 1964,
 1965,
 1966,
 1967,
 1968,
 1969,
 1970,
 1971,
 1972,
 1973,
 1974,
 1975,
 1976,
 1977,
 1978,
 1979,
 1980,
 1981,
 1982,
 1983,
 1984,
 1985,
 1986,
 1987,
 1988,
 1989,
 1990,
 1991,
 1992,
 1993,
 1994,
 1995,
 1996,
 1997,
 1998,
 1999,
 2000,
 2001,
 2002,
 2003,
 2004,
 2005,
 2006,
 2007,
 2008,
 2009,
 2010,
 2011,
 2012,
 2013,
 2014,
 2015,
 2016,
 2017,
 2018,
 2019,
 2020,
 2021]

In [13]:
emmy.nunique(axis=0)

id          21503
year           71
category     1333
nominee      5627
staff       15735
company       247
producer     1273
win             2
dtype: int64

In [14]:
emmy.isnull().sum()

id              0
year            0
category        0
nominee       156
staff          58
company       364
producer    13572
win             0
dtype: int64

In [15]:
emmy[emmy['company'] == 'Netflix']

,id,year,category,nominee,staff,company,producer,win
3,4,2019,Outstanding Character Voice-Over Performance,F Is For Family,"Kevin Michael Richardson, as Rosie",Netflix,Wild West Television in association with Gaumo...,False
6,7,2019,Outstanding Production Design For A Narrative ...,Ozark,"Derek R. Hill, Production Designer; John Richa...",Netflix,Media Rights Capital,False
7,8,2019,Outstanding Production Design For A Narrative ...,The Umbrella Academy,"Mark Worthington, Production Designer; Mark St...",Netflix,UCP,False
11,12,2019,Outstanding Production Design For A Narrative ...,A Series Of Unfortunate Events,"Bo Welch, Production Designer; Don MacAulay, A...",Netflix,A Netflix Original Production,False
17,18,2019,"Outstanding Production Design For A Variety, R...",Queer Eye,"Thomas Rouse, Production Designer",Netflix,"Scout Productions, Inc. and ITV Entertainment,...",False
19,20,2019,Outstanding Production Design For A Variety Sp...,Homecoming: A Film By Beyoncé,"Ric Lipson, Production Designer; Rachel Duncan...",Netflix,Parkwood Entertainment,False
23,24,2019,Outstanding Casting For A Comedy Series,Russian Doll,"Christine Kromer, CSA, Casting by",Netflix,Universal Television in association with Jax M...,False
26,27,2019,Outstanding Casting For A Drama Series,Ozark,"Alexa L. Fogel, CSA, Casting by; Tara Feldstei...",Netflix,Media Rights Capital,False
46,47,2019,Outstanding Cinematography For A Nonfiction Pr...,Our Planet,"Doug Anderson, Director of Photography; Gavin ...",Netflix,Silverback Films,False
47,48,2019,Outstanding Cinematography For A Nonfiction Pr...,Our Planet,"Alastair MacEwen, Director of Photography; Mat...",Netflix,Silverback Films,False


In [16]:
#performing checks to see the number of matches between emmy and movies. 
#movies.loc[movies['Title'].str.contains('Orange Is The New Black', na=False)]
#movies.loc[movies.originalTitle.duplicated(),['originalTitle']]
emmy.loc[((emmy['nominee'].isin(movies['Title'])) & (emmy['company'] == 'Netflix'))] 

,id,year,category,nominee,staff,company,producer,win
23,24,2019,Outstanding Casting For A Comedy Series,Russian Doll,"Christine Kromer, CSA, Casting by",Netflix,Universal Television in association with Jax M...,False
100,101,2019,Outstanding Single-Camera Picture Editing For ...,Russian Doll,"Laura Weinberg, Editor",Netflix,Universal Television in association with Jax M...,False
193,194,2019,Outstanding Lead Actress In A Comedy Series,Russian Doll,"Natasha Lyonne, as Nadia",Netflix,Universal Television in association with Jax M...,False
317,318,2019,Outstanding Stunt Coordination For A Comedy Se...,Russian Doll,"Christopher Place, Stunt Coordinator",Netflix,Universal Television in association with Jax M...,False
331,332,2019,Outstanding Writing For A Comedy Series,Russian Doll,"Leslye Headland, Teleplay by & Story by; Natas...",Netflix,Universal Television in association with Jax M...,False
332,333,2019,Outstanding Writing For A Comedy Series,Russian Doll,"Allison Silverman, Written by",Netflix,Universal Television in association with Jax M...,False
335,336,2019,Outstanding Writing For A Drama Series,Bodyguard,"Jed Mercurio, Written by",Netflix,World Productions,False
348,349,2019,Outstanding Writing For A Variety Special,Adam Sandler: 100% Fresh,"Adam Sandler, Written by",Netflix,Irwin Entertainment l Happy Madison Productions,False
352,353,2019,Outstanding Writing For A Variety Special,Wanda Sykes: Not Normal,"Wanda Sykes, Written by",Netflix,Push It Productions,False
377,378,2019,Outstanding Music Supervision,Russian Doll,"Brienne Rose, Music Supervisor",Netflix,Universal Television in association with Jax M...,False


In [17]:
movies.loc[movies['Title'].str.contains('Barry', na=False)]

,Unnamed: 0,ID,Title,Year,Rating,IMDb,Rotten Tomatoes,Genre,Netflix,Amazon Prime Video
896,896,897,Sunshine Barry & the Disco Worms,2008,7+,4.6,27,Family,0,1
2428,2428,2429,Someone Marry Barry,2014,NaN,5.8,57,Comedy,0,1
2869,2869,2870,Sunshine Barry & the Disco Worms,2008,7+,4.6,27,Animation,0,1
8780,8780,8781,The Nine Lives of Marion Barry,2009,7+,7.1,34,Documentary,0,1
14477,14477,14478,Sunshine Barry & the Disco Worms,2008,7+,4.6,27,Comedy,0,1
15428,15428,15429,The Adventures of Barry McKenzie,1972,18+,5.7,39,Comedy,0,1
18526,18526,18527,"Barry Brewer: Chicago, I'm Home",2019,NaN,7.2,39,Comedy,0,1
22320,22320,22321,Barry,2016,NaN,5.8,66,Drama,1,0
22941,22941,22942,Todd Barry: Spicy Honey,2017,16+,6.8,41,Comedy,1,0


In [18]:
show_names = emmy.loc[emmy['nominee'].isin(movies['Title']),'nominee'].unique()
#show_names = emmy['nominee'].unique()


In [19]:
show_names.size

143

In [20]:
emmy.loc[~emmy['nominee'].isin(movies['Title']),'nominee'].unique()

array(['The Simpsons', 'Family Guy',
       'When You Wish Upon A Pickle: A Sesame Street Special', ...,
       'Louis McManus For His Original Design of the Emmy',
       'KTLA for Outstanding overall achievement in 1948',
       'Charles Mesak/Don Lee TV for Phasefader - In Recog'], dtype=object)

In [21]:
movies.loc[movies['Title'].str.contains('Family Guy', na=False)]

,Unnamed: 0,ID,Title,Year,Rating,IMDb,Rotten Tomatoes,Genre,Netflix,Amazon Prime Video


In [22]:
movies_b.loc[movies_b['Title'].str.contains('Family Guy', na=False)]

,Unnamed: 0,ID,Title,Year,Age,Rotten Tomatoes,Netflix,Hulu,Prime Video,Disney+,Type


In [70]:
#Checking here to see if the second movie dataset that we identified has more matches with the emmy.
#its about the same. Out of 5627 titles, there are only about 140 titles for there is a match in emmy dataset
#see below the number of titles that do not have a match with the second movie dataset
emmy.loc[~emmy['nominee'].isin(movies_b['Title']),'nominee'].unique().size

5530

In [71]:
emmy.loc[~emmy['nominee'].isin(movies['Title']),'nominee'].unique().size

5485

In [24]:
oscar.nunique(axis=0)

year_film          92
year_ceremony      92
ceremony           92
category          111
name             6666
film             4833
winner              2
dtype: int64

In [25]:
oscar.isnull().sum()

year_film          0
year_ceremony      0
ceremony           0
category           0
name               0
film             304
winner             0
dtype: int64

In [68]:
#Title mismatches (out of 4833) with the old movies dataset
oscar.loc[~oscar['film'].isin(movies['Title']),'film'].unique().size

4389

In [69]:
#Title mismatches with the new movies dataset
oscar.loc[~oscar['film'].isin(movies_b['Title']),'film'].unique().size

4285

In [27]:
#oscar.loc[oscar['film'] =='Terminator: Dark Fate']
oscar.loc[oscar['film'].str.contains('Terminator', na=False)]

,year_film,year_ceremony,ceremony,category,name,film,winner
7025,1991,1992,64,CINEMATOGRAPHY,Adam Greenberg,Terminator 2: Judgment Day,False
7050,1991,1992,64,FILM EDITING,"Conrad Buff, Mark Goldblatt, Richard A. Harris",Terminator 2: Judgment Day,False
7059,1991,1992,64,MAKEUP,"Stan Winston, Jeff Dawn",Terminator 2: Judgment Day,True
7085,1991,1992,64,SOUND,"Tom Johnson, Gary Rydstrom, Gary Summers, Lee ...",Terminator 2: Judgment Day,True
7088,1991,1992,64,SOUND EFFECTS EDITING,"Gary Rydstrom, Gloria S. Borders",Terminator 2: Judgment Day,True
7091,1991,1992,64,VISUAL EFFECTS,"Dennis Muren, Stan Winston, Gene Warren, Jr., ...",Terminator 2: Judgment Day,True


In [28]:
#problem? what does the movie database have? What does popular mean?
#movies.loc[movies['Title'] =='Terminator: Judgment Day']
movies.loc[movies['Title'].str.contains('Terminator', na=False)]

,Unnamed: 0,ID,Title,Year,Rating,IMDb,Rotten Tomatoes,Genre,Netflix,Amazon Prime Video
0,0,1,Terminator: Dark Fate,2019,18+,6.2,81,Action & Adventure,0,1


In [29]:
#performing checks on unique values. note that the title fields have duplicates
title_info.nunique(axis=0)

tconst            112356
titleType             10
primaryTitle       13151
originalTitle      12673
isAdult                4
startYear            205
endYear               76
runtimeMinutes       284
genres              1302
dtype: int64

In [30]:
title_info.isnull().sum()

tconst            0
titleType         0
primaryTitle      0
originalTitle     0
isAdult           0
startYear         0
endYear           0
runtimeMinutes    0
genres            0
dtype: int64

In [31]:
title_info.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
178,tt0000181,short,Cinderella,Cinderella,0,1898,\N,\N,"Fantasy,Short"
305,tt0000308,short,The House That Jack Built,The House That Jack Built,0,1900,\N,1,Short
359,tt0000363,short,Magical Sword,The Magic Sword,0,1901,\N,2,"Fantasy,Romance,Short"
395,tt0000399,short,Jack and the Beanstalk,Jack and the Beanstalk,0,1902,\N,10,"Fantasy,Short"
561,tt0000565,short,The Night Before Christmas,The Night Before Christmas,0,1905,\N,9,"Animation,Family,Fantasy"


In [32]:
title_info[(title_info['originalTitle'] == 'Cinderella') &(title_info['startYear'] == '1965')]
#.groupby(['titleType','startYear']).count()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
2712508,tt13154228,tvMovie,Cinderella,Cinderella,0,1965,\N,120,Musical


In [33]:
movies[movies['Title'] == 'Cinderella']

,Unnamed: 0,ID,Title,Year,Rating,IMDb,Rotten Tomatoes,Genre,Netflix,Amazon Prime Video
884,884,885,Cinderella,1965,all,7.8,57,Family,0,1


In [34]:
title_info.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 112356 entries, 178 to 8541347
Data columns (total 9 columns):
tconst            112356 non-null object
titleType         112356 non-null object
primaryTitle      112356 non-null object
originalTitle     112356 non-null object
isAdult           112356 non-null object
startYear         112356 non-null object
endYear           112356 non-null object
runtimeMinutes    112356 non-null object
genres            112356 non-null object
dtypes: object(9)
memory usage: 8.6+ MB


In [35]:
#remove titles before the year 1912. Replace first \N with nan value
title_info = title_info.replace('\\N',np.NaN)
title_info.head()


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
178,tt0000181,short,Cinderella,Cinderella,0,1898,NaN,NaN,"Fantasy,Short"
305,tt0000308,short,The House That Jack Built,The House That Jack Built,0,1900,NaN,1,Short
359,tt0000363,short,Magical Sword,The Magic Sword,0,1901,NaN,2,"Fantasy,Romance,Short"
395,tt0000399,short,Jack and the Beanstalk,Jack and the Beanstalk,0,1902,NaN,10,"Fantasy,Short"
561,tt0000565,short,The Night Before Christmas,The Night Before Christmas,0,1905,NaN,9,"Animation,Family,Fantasy"


In [36]:
#Remove entries for years less than 1912, genres that are talkshows, and titles that are short, videos and tvspecials
#also removed tvepisode since they didn't seem relevent for our analysis. 
title_info_clean = title_info[((title_info['startYear'].astype(np.float)>=1912) 
                               & (title_info['genres'].str.contains('Talk-Show') == False)
                               & (title_info['titleType'].isin(['short','video','tvShort','tvEpisode', 'tvSpecial', 'tvMiniSeries', 'videoGame']) == False)
                              )]

In [37]:
title_info_clean.titleType.unique()

array(['movie', 'tvMovie', 'tvSeries'], dtype=object)

In [38]:
movies['Year'].dtypes

dtype('int64')

In [39]:
#After removing the year, the duplicates reduced from 68716 to 50444
#but if you jus check the unique values, there are about 28470
#title_info_clean.loc[title_info_clean.originalTitle.duplicated(),['originalTitle']].originalTitle.unique().size
title_info_clean.loc[title_info_clean.originalTitle.duplicated(),['originalTitle']].originalTitle.unique


<bound method Series.unique of 2379                Oliver Twist
5859       The Prisoner of Zenda
6908                     Macbeth
7051                Oliver Twist
7360        The Three Musketeers
                   ...          
8529332                 The Line
8535270               The System
8535495              Good Enough
8537956               Blown Away
8540312                 Paradise
Name: originalTitle, Length: 11590, dtype: object>

In [40]:
#I checked a couple of them and it the duplicates seem to be legitimate (Oliver Twist for example)
#The other point is that the movies database doesn't seem to be comprehesive enough. 
#also remove entries which are talk shows. 
#Strange that some of the movies are not in this file downloaded from IMDB: Along with the Gods: The Two Worlds, The First King 
#Though you can find IMDB ratings for them when you search for the movie names. 
title_info.loc[title_info['originalTitle'].str.contains('I Am Jonas', na=False)]
#title_info_clean.loc[title_info_clean['originalTitle'].str.contains('Along with the Gods: The Two Worlds', na=False)]
#title_info_clean.loc[((title_info_clean['originalTitle'].str.contains('Kill Chain', na=False)) & (title_info_clean['startYear'] == '2019'))]
#movies.loc[movies['Title'].str.contains('Oliver Twist', na=False)]

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres


In [41]:
title_info[['titleType','genres']].groupby(['titleType','genres']).head()

,titleType,genres
178,short,"Fantasy,Short"
305,short,Short
359,short,"Fantasy,Romance,Short"
395,short,"Fantasy,Short"
561,short,"Animation,Family,Fantasy"
578,short,"Drama,Short"
605,short,"Drama,Romance,Short"
630,short,Short
647,short,"Action,Short"
661,short,"Action,Drama,Short"


In [42]:
movie_names = oscar.loc[oscar['film'].isin(movies['Title']),'film'].unique()

In [43]:
movie_names.size

445

In [44]:
movies['type'] = np.nan

In [45]:
#adding the type column
movies.loc[movies['Title'].isin(show_names), 'type'] = 'shows'
movies.loc[movies['Title'].isin(movie_names), 'type'] = 'movie'

In [46]:
movies.head(100)

,Unnamed: 0,ID,Title,Year,Rating,IMDb,Rotten Tomatoes,Genre,Netflix,Amazon Prime Video,type
0,0,1,Terminator: Dark Fate,2019,18+,6.2,81,Action & Adventure,0,1,NaN
1,1,2,Gemini Man,2019,13+,5.7,74,Action & Adventure,0,1,NaN
2,2,3,Rambo: Last Blood,2019,18+,6.1,72,Action & Adventure,0,1,NaN
3,3,4,The Courier,2019,18+,4.9,50,Action & Adventure,0,1,NaN
4,4,5,Crawl,2019,18+,6.1,79,Action & Adventure,0,1,NaN
5,5,6,The Kill Team,2019,18+,5.9,61,Action & Adventure,0,1,NaN
6,6,7,The Rhythm Section,2020,18+,5.3,63,Action & Adventure,0,1,NaN
7,7,8,Legionnaire's Trail,2020,18+,3.1,39,Action & Adventure,0,1,NaN
8,8,9,The Avengers,2012,13+,8.0,94,Action & Adventure,0,1,shows
9,9,10,Escape Plan: The Extractors,2019,18+,4.4,49,Action & Adventure,0,1,NaN


In [47]:
movie_names_null = movies.loc[movies['type'].isnull(),'Title']

In [48]:
#since there are duplicate movie names with some being movies and other shows. Take the example of Oliver Twist above. 
#To exactly match the type, we also need to match the year of the movie/shows

title_info_clean_small = title_info_clean[['titleType','originalTitle','startYear']]
title_info_clean_small = title_info_clean_small[title_info_clean_small['originalTitle'].isin(movie_names_null)]
title_info_clean_small = title_info_clean_small.drop_duplicates(
  subset = ['originalTitle', 'startYear','titleType']).reset_index(drop = True)
title_info_clean_small = title_info_clean_small.rename(columns={"originalTitle": "Title","titleType":"type","startYear":"Year"})


In [49]:
title= 'Terminator: Dark Fate'
year = 2019
title_info_clean.loc[((title_info_clean['originalTitle']== title) & (title_info_clean['startYear'].astype(int) == year)),'titleType']

6951218    movie
Name: titleType, dtype: object

In [50]:
title= 'Danger Close'
year = 2019

stype = title_info_clean_small.loc[((title_info_clean_small['Title']== title) & (title_info_clean_small['Year'].astype(int) == year)),'type']
#stype = title_info_clean.loc[((title_info_clean['originalTitle']== title) & (title_info_clean['startYear'] == year)),'titleType']
# stype = type_amazon.loc[((type_amazon['title']== title) & (type_amazon['release_year'].astype(int) == year)),'type']
# print(stype)
if(stype.empty == False):
    print(stype.values.size)
    print(title + ": count -" + str(stype.values.size) + str(stype.values[0]))
elif((type_amazon.loc[((type_amazon['title']== title) & (type_amazon['release_year'].astype(int) == year)),'type']).empty == False):
    stype = type_amazon.loc[((type_amazon['title']== title)),'type']
    print(stype.values[0].lower())
elif((type_netflix.loc[((type_netflix['title']== title) & (type_netflix['release_year'].astype(int) == year)),'type']).empty == False):
    stype = type_netflix.loc[((type_netflix['title']== title)),'type']
    print(stype.values[0])
else:
     print("no value")

movie


In [51]:
#movies["Type"]
def get_type(title, year,oldtype):
    stype = title_info_clean_small.loc[((title_info_clean_small['Title']== title) & (title_info_clean_small['Year'].astype(int) == year)),'type']
    #print(title)
    #print(year)
    val =""
    if(stype.empty == False):
        #if(stype.values.size > 1):
            #print(title + " " + str(year) + ": count -" + str(stype.values.size))
        val = stype.values[0]
    #not matching years for the below as in some cases there are year mismatches
    elif((type_amazon.loc[((type_amazon['title']== title)),'type']).empty == False):
        stype = type_amazon.loc[((type_amazon['title']== title)),'type']
        val= stype.values[0].lower()
    elif((type_netflix.loc[((type_netflix['title']== title)),'type']).empty == False):
        stype = type_netflix.loc[((type_netflix['title']== title)),'type']
        val = stype.values[0].lower()
    else:
        val = oldtype
    if(val == 'tv show'):
        val = "shows"
    return val
movies['type']= movies.apply(lambda x: get_type(x.Title, x.Year, x.type), axis=1)


In [52]:
#prior to mapping with IMDB database, the null type values were 23687
movies.isnull().sum()

Unnamed: 0                0
ID                        0
Title                     0
Year                      0
Rating                14635
IMDb                    420
Rotten Tomatoes           0
Genre                     0
Netflix                   0
Amazon Prime Video        0
type                   6012
dtype: int64

In [53]:
#271304
movies.size

271304

In [54]:
#These movie don't seem to be a match in any of the other files. 
movies[movies['type'].isnull() == True]

,Unnamed: 0,ID,Title,Year,Rating,IMDb,Rotten Tomatoes,Genre,Netflix,Amazon Prime Video,type
23,23,24,The First King,2019,18+,6.5,54,Action & Adventure,0,1,NaN
26,26,27,Along with the Gods: The Two Worlds,2017,7+,7.3,73,Action & Adventure,0,1,NaN
33,33,34,Asterix the Gaul,1967,all,6.6,53,Animation,0,1,NaN
34,34,35,SlugTerra: Return of the Elementals,2014,7+,6.4,48,Animation,0,1,NaN
35,35,36,Ooops! Noah is Gone...,2015,all,5.8,57,Animation,0,1,NaN
38,38,39,The Stolen Princess: Ruslan and Ludmila,2018,NaN,6.1,56,Animation,0,1,NaN
40,40,41,Donkey X,2007,7+,4.6,32,Animation,0,1,NaN
47,47,48,Top Cat Begins,2015,7+,4.2,38,Animation,0,1,NaN
57,57,58,Flight of the Butterflies,2012,all,7.0,66,Biography,0,1,NaN
60,60,61,The King's Choice,2016,NaN,7.1,75,Biography,0,1,NaN


In [55]:
#don't think there needs to be a distinction between movie and TV movie, so setting all tvmovie to movie
movies[movies['type'] == "tvMovie"]

,Unnamed: 0,ID,Title,Year,Rating,IMDb,Rotten Tomatoes,Genre,Netflix,Amazon Prime Video,type
25,25,26,The Last Sharknado: It's About Time,2018,16+,3.5,47,Action & Adventure,0,1,tvMovie
146,146,147,Python,2000,18+,3.7,31,Comedy,0,1,tvMovie
154,154,155,An Inspector Calls,2015,7+,7.7,66,Crime,0,1,tvMovie
224,224,225,Monsterwolf,2010,18+,3.6,32,Documentary,0,1,tvMovie
291,291,292,Stalked by My Neighbor,2015,16+,5.2,40,Drama,0,1,tvMovie
301,301,302,Menace from Outer Space,1956,NaN,3.7,34,Family,0,1,tvMovie
315,315,316,The Rooftop Christmas Tree,2016,all,6.2,41,Family,0,1,tvMovie
329,329,330,Out There,1995,13+,5.5,42,Family,0,1,tvMovie
340,340,341,Witch's Night Out,1978,NaN,7.5,48,Fantasy,0,1,tvMovie
354,354,355,Little Spirit: Christmas in New York,2008,all,5.0,36,Fantasy,0,1,tvMovie


In [56]:
movies['type'].loc[(movies['type'] == "tvMovie")]= 'movie'
movies['type'].loc[(movies['type'] == "tvSeries")]= 'shows'

In [57]:
movies.type.unique()

array(['movie', 'shows', nan], dtype=object)

In [58]:
movies[movies['Amazon Prime Video']==1].groupby(['Amazon Prime Video','type']).agg('count')

Unnamed: 0     ID  Title   Year  Rating   IMDb  \
Amazon Prime Video type                                                    
1                  movie       14857  14857  14857  14857    6052  14695   
                   shows         236    236    236    236     105    230   

                          Rotten Tomatoes  Genre  Netflix  
Amazon Prime Video type                                    
1                  movie            14857  14857    14857  
                   shows              236    236      236

In [59]:
movies[movies['Netflix']==1].groupby(['Netflix','type']).agg('count')

Unnamed: 0    ID  Title  Year  Rating  IMDb  Rotten Tomatoes  \
Netflix type                                                                  
1       movie        3516  3516   3516  3516    1758  3478             3516   
        shows          43    43     43    43      20    41               43   

               Genre  Amazon Prime Video  
Netflix type                              
1       movie   3516                3516  
        shows     43                  43

In [60]:
movies_b.nunique(axis=0)

Unnamed: 0         9515
ID                 9515
Title              9515
Year                103
Age                   5
Rotten Tomatoes      85
Netflix               2
Hulu                  2
Prime Video           2
Disney+               2
Type                  1
dtype: int64

In [61]:
movies_b.head()

,Unnamed: 0,ID,Title,Year,Age,Rotten Tomatoes,Netflix,Hulu,Prime Video,Disney+,Type
0,0,1,The Irishman,2019,18+,98/100,1,0,0,0,0
1,1,2,Dangal,2016,7+,97/100,1,0,0,0,0
2,2,3,David Attenborough: A Life on Our Planet,2020,7+,95/100,1,0,0,0,0
3,3,4,Lagaan: Once Upon a Time in India,2001,7+,94/100,1,0,0,0,0
4,4,5,Roma,2018,18+,94/100,1,0,0,0,0


In [62]:
movies_b.groupby(['Netflix','Prime Video']).agg('count')

Unnamed: 0    ID  Title  Year   Age  Rotten Tomatoes  \
Netflix Prime Video                                                         
0       0                  1816  1816   1816  1816  1341             1816   
        1                  4004  4004   4004  4004  2099             4004   
1       0                  3586  3586   3586  3586  1832             3579   
        1                   109   109    109   109    66              109   

                     Hulu  Disney+  Type  
Netflix Prime Video                       
0       0            1816     1816  1816  
        1            4004     4004  4004  
1       0            3586     3586  3586  
        1             109      109   109

In [63]:
movies_b.Year.unique()

array([2019, 2016, 2020, 2001, 2018, 2017, 2004, 2014, 2015, 2006, 1991,
       2012, 2010, 2011, 2013, 2005, 1998, 2002, 2009, 1971, 2003, 2007,
       1975, 1988, 2008, 1974, 1979, 1997, 1990, 2021, 1984, 2000, 1993,
       1994, 1999, 1976, 1964, 1954, 1982, 1986, 1992, 1995, 1987, 1989,
       1980, 1996, 1958, 1981, 1977, 1983, 1970, 1963, 1985, 1972, 1962,
       1966, 1956, 1969, 1973, 1960, 1959, 1978, 1968, 1940, 1941, 1965,
       1923, 1950, 1953, 1957, 1955, 1946, 1936, 1952, 1967, 1935, 1951,
       1925, 1947, 1945, 1948, 1939, 1942, 1932, 1937, 1919, 1949, 1934,
       1931, 1922, 1927, 1933, 1920, 1961, 1944, 1943, 1915, 1929, 1938,
       1914, 1930, 1928, 1916])

In [64]:
#Titles that are not in the the first movie dataset
movies_b.loc[~movies_b['Title'].isin(movies['Title'])]

,Unnamed: 0,ID,Title,Year,Age,Rotten Tomatoes,Netflix,Hulu,Prime Video,Disney+,Type
0,0,1,The Irishman,2019,18+,98/100,1,0,0,0,0
2,2,3,David Attenborough: A Life on Our Planet,2020,7+,95/100,1,0,0,0,0
5,5,6,To All the Boys I've Loved Before,2018,13+,94/100,1,0,0,0,0
6,6,7,The Social Dilemma,2020,13+,93/100,1,0,0,0,0
8,8,9,The Ballad of Buster Scruggs,2018,16+,92/100,1,0,0,0,0
9,9,10,The Trial of the Chicago 7,2020,18+,92/100,1,0,0,0,0
12,12,13,Dolemite Is My Name,2019,18+,92/100,1,0,0,0,0
15,15,16,Fyre,2019,18+,91/100,1,0,0,0,0
16,16,17,Miss Americana,2020,18+,90/100,1,0,0,0,0
17,17,18,Virunga,2014,16+,90/100,1,0,0,0,0


In [65]:
#Next steps
#Movies master database: Fill gaps in Type, Rating; nominated (boolean)/ no.of nominations/ no. of winners
#Oscar and the emmy: 
### number of awards won for each show/movie; 
#? prep for analysis
#sub dataframes or files (aggregtes over time - production year and the years in award files)
